## **Import statements**

In [6]:
%reload_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import os

from azureml.core import Workspace
 
#ws = Workspace.create(name='neusomaticws', subscription_id='3d890015-29d9-4a08-bad3-8e41a8c8c199',
#                      resource_group='hpc-roche-neusomatic',
#                      exist_ok=True,
#                      location='westeurope' 
#                     )
#ws.get_details()

In [7]:
# Create the configuration file.
#ws.write_config() #config is saved in .azureml/config.json

# Use this code to load the workspace from 
# other scripts and notebooks in this directory.
ws = Workspace.from_config()
ws.get_details()

{'id': '/subscriptions/3d890015-29d9-4a08-bad3-8e41a8c8c199/resourceGroups/hpc-roche-neusomatic/providers/Microsoft.MachineLearningServices/workspaces/neusomaticws',
 'name': 'neusomaticws',
 'location': 'westeurope',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'workspaceid': 'a1186608-73dc-4c0e-a933-5255238b786a',
 'description': '',
 'friendlyName': '',
 'creationTime': '2019-01-17T14:24:40.4006018+00:00',
 'containerRegistry': '/subscriptions/3d890015-29d9-4a08-bad3-8e41a8c8c199/resourcegroups/hpc-roche-neusomatic/providers/microsoft.containerregistry/registries/neusomaticws9296802368',
 'keyVault': '/subscriptions/3d890015-29d9-4a08-bad3-8e41a8c8c199/resourcegroups/hpc-roche-neusomatic/providers/microsoft.keyvault/vaults/neusomaticws9199451487',
 'applicationInsights': '/subscriptions/3d890015-29d9-4a08-bad3-8e41a8c8c199/resourcegroups/hpc-roche-neusomatic/providers/microsoft.insights/components/neusomaticws7558457736',
 'identityPrincipalId': '99ca2d15-e5da-4a68-bbf6

## **Configure the Azure ML Compute and Storage**

In [4]:
# get azure storage key
#Assuming there is a single line with a key  
def get_storage_key(file):
    File = open(file, 'r', 0)
    return File.readlines()[0]
    

In [6]:
#list all datastores registered in current workspace
datastores = ws.datastores
for name, ds in datastores.items():
    print(name, ds.datastore_type)
    
#print(ds.datastore_type, ds.account_name, ds.container_name)



workspacefilestore AzureFile
workspaceblobstore AzureBlob
ds_neus AzureBlob
ds_test AzureBlob
ds_res AzureBlob


In [7]:
from azureml.core import Workspace, Datastore

#get named datastore from current workspace
#ds_weights = Datastore.get(ws, datastore_name='ds_weights')
#ds_tracks = Datastore.get(ws, datastore_name='ds_tracks')
ds_test = Datastore.get(ws, datastore_name='ds_test')

## **Set Compute Resources in Azure**

Following options are available:
* **Run-based creation** creates a compute target at runtime. The compute is automatically created for your run. The cluster scales up to the number of max_nodes that you specify in your run config. The compute is deleted automatically once the run completes.

* **Persistent Compute**. A persistent Azure Machine Learning Compute can be reused across jobs. The compute can be shared with other users in the workspace and is kept between jobs.

* **Remote Virtual Machines**. An arbitrary remote VM, as long as it's accessible from Azure Machine Learning service.

* **Azure Batch**. Azure Batch is used to run large-scale parallel and high-performance computing (HPC) applications efficiently in the cloud. AzureBatchStep can be used in an Azure Machine Learning Pipeline to submit jobs to an Azure Batch pool of machines. 

See https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-set-up-training-targets#vm for more details.


#### Option #4.  Create a Data Science VM 
* VM has many dependencies as a part of the OS image

Testing now.  


In [8]:
from azureml.core.compute import DsvmCompute
import time

dsvm_name = 'dsvmgpu'
try:
    dsvm_compute = DsvmCompute(ws, dsvm_name)
    print('Found an existing DSVM.')
except:
    print('Creating a new DSVM.')
    dsvm_config = DsvmCompute.provisioning_configuration(vm_size = "Standard_NC6") # Standard_D2s_v3
    dsvm_compute = DsvmCompute.create(ws, name = dsvm_name, provisioning_configuration = dsvm_config)
    dsvm_compute.wait_for_completion(show_output = True)
    print("Waiting one minute for ssh to be accessible")
    time.sleep(90) # Wait for ssh to be accessible

compute_target = dsvm_compute

Creating a new DSVM.

        DEPRECATED
        This class will be deprecated soon and we will remove support for it in an upcoming release.
        Please use the "AmlCompute" class instead, or spin up a VM in Azure and attach it using RemoteCompute().
        Use !help AmlCompute to learn more.
        
Creating..........................................
SucceededProvisioning operation finished, operation "Succeeded"
Waiting one minute for ssh to be accessible


## Execution Control 

#### Option 1: Execute with Run Configurations (skip for now).
* execute once the compute resource is defined
* Select between Options 1-4
* Define conda and pip dependencies






Option 2: Define Estimators
* execute once the compute resource is defined
* Select between Options 1-4
* Define conda and pip dependencies





In [9]:
from azureml.core import Workspace, Experiment, Run
from azureml.train.estimator import Estimator

#Original script is called as follows:
# preprocess.py --mode train 
#--reference GRCh38.fa \
#--region_bed region.bed \
#--tumor_bam tumor.bam \
#--normal_bam normal.bam \
#--work work_train \
#--truth_vcf truth.vcf \
#--min_mapq 10 \
#--number_threads 10 \
#--scan_alignments_binary ../bin/scan_alignments

#conda install zlib=1.2.11 numpy=1.15.4 scipy=1.2.0 cmake=3.13.2 imageio=2.5.0
#$conda install pysam=0.15.2 pybedtools=0.8.0 samtools=1.9 tabix=0.2.6 bedtools=2.27.1 biopython=1.73 -c bioconda
#$conda install pytorch=1.0.1 torchvision=0.2.1 cudatoolkit=8.0 -c pytorch

script_params = {
    '--reference': ds_rphi.as_download(),
    '--work': 'work_train',
    '--region_bed': ds_weights.as_download(),
    '--truth-tracks-path': ds_tracks.as_download()
}

# compute_target is a CPU cluster
# my_compute_target is a default compute from AzureML (not tested)
est = Estimator(source_directory=source_folder,
                script_params=script_params,
                compute_target=compute_target,  
                entry_script='train.py',
                #pip_packages=['pynvrtc'],
                conda_packages=['pytorch','scipy','numpy','scikit-learn','tqdm','cython','torchvision','cudatoolkit=9.0','matplotlib'])

#ImportError: .../torch/lib/libtorch.so.1: undefined symbol: nvrtcGetProgramLogSize
#Testing pytorch-cpu version -> problem 
#ResolvePackageNotFound: torchvision-cpu, pytorch-cpu

#OR add cudatoolkit

# Create the experiment
experiment = Experiment(workspace = ws, name = "Neusomatic-GPU")
#my_compute_target = run_system_managed_temp.amlcompute

run = experiment.submit(est)
run.wait_for_completion(show_output = True)

#run_system_managed_temp = experiment.start_logging()

IndentationError: unexpected indent (<ipython-input-9-b0c7d1cb3bd9>, line 6)

In [ ]:
run
#run.log(name, value, description='')

# ImportError: /azureml-envs/azureml_aa3800697c3da90356674c5d162cf01e/lib/python3.6/site-packages/torch/lib/libtorch.so.1: undefined symbol: nvrtcGetProgramLogSize
